In [1]:
import pandas as pd
from time import time
from sklearn.model_selection import cross_val_score
from scipy import stats
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# global variable for storing models
models = []

In [3]:
# to read json file
def to_read_json(filename,title):
    data = pd.read_json(filename,orient='columns')
    data_format = pd.read_json(([title]).to_json(),orient='index')
    return data_format

In [4]:
# to read csv file
def to_read_csv(csvfile):
    df = pd.read_csv(csvfile,index_col=None,header=0)
    return df

In [5]:
#to make directory
def make_directory():
    try:
        if not os.path.exists('ADS_Assignment3'):
            os.makedirs('ADS_Assignment3', mode=0o777)
        else:
            shutil.rmtree(os.path.join(os.path.dirname(__file__),'ADS_Assignment3'),ignore_errors=False)
            os.makedirs('ADS_Assignment3', mode=0o777)
    except Exception as e:
        exit()

In [6]:
# Data_standardization
from sklearn import preprocessing
def data_standardization(X):
    standardized_X = pd.DataFrame(preprocessing.scale(X),index =X.index,columns = X.columns)
    return standardized_X

In [7]:
# Scaling
from sklearn.preprocessing import MinMaxScaler
def minmaxscaling(X):
    min_max=MinMaxScaler()
    X_minmax=pd.DataFrame(min_max.fit_transform(X),index =X.index,columns = X.columns)
    return X_minmax

In [8]:
# to define train test split
from sklearn.model_selection import train_test_split
def train_test_split_model(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.30, random_state=42)
    return X_train,X_test,y_train,y_test

In [ ]:
def calculate_recall_precision(name,label, prediction):
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0
    # test
    for i in range(0, len(label)):
        if prediction[i] == 1:
            if prediction[i] == label[i]:
                true_positives += 1
            else:
                false_positives += 1
        else:
            if prediction[i] == label[i]:
                true_negatives += 1
            else:
                false_negatives += 1
                
    print('TruePositive_test ',true_positives)
    print('TrueNegative_test ',true_negatives)
    print('FlasePositive_test ',false_positives)
    print('FalseNegative_test ',false_negatives)  
    # a ratio of correctly predicted observation to the total observations
    accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)

    # precision is "how useful the search results are"
    try :
        precision = true_positives / (true_positives + false_positives)
        
    # recall is "how complete the results are"
    recall = true_positives / (true_positives + false_negatives)
    #f1_score
    f1_score = 2 / ((1 / precision) + (1 / recall))
    #specificity
    specificity = true_negatives/((true_negatives + false_positives))
    
    df_local = pd.DataFrame({'Model':[name],'Precision': [precision],
                             'Recall': [recall],'F1_score':[f1_score],
                             'Accuracy': [accuracy],'Specificity': [specificity]})
    error_metric = pd.concat([df_local])
 
    return error_metric

In [9]:
def calculate_recall_precision(name,label, prediction):
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0
    # test
    for i in range(0, len(label)):
        if prediction[i] == 1:
            if prediction[i] == label[i]:
                true_positives += 1
            else:
                false_positives += 1
        else:
            if prediction[i] == label[i]:
                true_negatives += 1
            else:
                false_negatives += 1
                
    print('TruePositive_test ',true_positives)
    print('TrueNegative_test ',true_negatives)
    print('FlasePositive_test ',false_positives)
    print('FalseNegative_test ',false_negatives)  
    # a ratio of correctly predicted observation to the total observations
    accuracy = (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)

    # precision is "how useful the search results are"
    precision = true_positives / (true_positives + false_positives)
    # recall is "how complete the results are"
    recall = true_positives / (true_positives + false_negatives)
    #f1_score
    f1_score = 2 / ((1 / precision) + (1 / recall))
    #specificity
    specificity = true_negatives/((true_negatives + false_positives))
    
    df_local = pd.DataFrame({'Model':[name],'Precision': [precision],
                             'Recall': [recall],'F1_score':[f1_score],
                             'Accuracy': [accuracy],'Specificity': [specificity]})
    error_metric = pd.concat([df_local])
 
    return error_metric

In [10]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
def logistic_regression_model(X,y):
    X_train,X_test,y_train,y_test = train_test_split_model(X,y)
    t1=time()
    lr = LogisticRegression()
    scores = cross_val_score(lr,X,y, cv=20)
    scores1 = scores.mean()
    deviation = scores.std() * 2 
    lr.fit(X_train,y_train)
    models.append(lr)
    log = calculate_recall_precision('LogisticRegression',y_test,lr.predict(X_test))
    t2=time()
    t = t2-t1
    log['Accuract with cv'] = scores1
    log['Deviation(+/-)'] = deviation
    log['time'] = t
    return log

In [11]:
# Naive Bayers
from sklearn.naive_bayes import GaussianNB
def naive_bayers_model(X,y):
    X_train,X_test,y_train,y_test = train_test_split_model(X,y)
    t1=time()
    nb = GaussianNB()
    scores = cross_val_score(nb,X,y, cv=20)
    scores1 = scores.mean()
    deviation = scores.std() * 2
    nb.fit(X_train,y_train)
    models.append(nb)
    naive = calculate_recall_precision('NaiveBayers',y_test,nb.predict(X_test))
    t2=time()
    t = t2-t1
    naive['Accuract with cv'] = scores1
    naive['Deviation(+/-)'] = deviation
    naive['time'] = t
    return naive

In [12]:
# Support Vector Machine
from sklearn.svm import SVC
def support_vector_machine_model(X,y):
    X_train,X_test,y_train,y_test = train_test_split_model(X,y)
    t1=time()
    svm = SVC(kernel="linear",C=0.025,random_state=101)
    scores = cross_val_score(svm,X,y, cv=20)
    scores1 = scores.mean()
    deviation = scores.std() * 2
    svm.fit(X_train,y_train)
    models.append(svm)
    svmachine =calculate_recall_precision('SupportVectorMachine',y_test,svm.predict(X_test))
    t2=time()
    t = t2-t1
    svmachine['Accuract with cv'] = scores1
    svmachine['Deviation(+/-)'] = deviation
    svmachine['time'] = t
    return svmachine

In [13]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
def DecisionTreeClassifier_model(X,y):
    X_train,X_test,y_train,y_test = train_test_split_model(X,y)
    t1=time()
    dtree = DecisionTreeClassifier(max_depth=10,random_state=101,max_features= None, min_samples_leaf = 15)
    scores = cross_val_score(dtree,X,y, cv=20)
    scores1 = scores.mean()
    deviation = scores.std() * 2
    dtree.fit(X_train,y_train)
    models.append(dtree)
    decision = calculate_recall_precision('DecisionTree',y_test,dtree.predict(X_test))
    t2=time()
    t = t2-t1
    decision['Accuract with cv'] = scores1
    decision['Deviation(+/-)'] = deviation
    decision['time'] = t
    return decision

In [14]:
# AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
def AdaBoostClassifier_model(X,y):
    X_train,X_test,y_train,y_test = train_test_split_model(X,y)
    t1=time()
    clf = AdaBoostClassifier(n_estimators=100, learning_rate=1)
    scores = cross_val_score(clf,X,y, cv=20)
    scores1 = scores.mean()
    deviation = scores.std() * 2
    clf.fit(X_train,y_train)
    models.append(clf)
    ada = calculate_recall_precision('AdaptiveBoosting',y_test,clf.predict(X_test))
    t2=time()
    t = t2-t1
    ada['Accuract with cv'] = scores1
    ada['Deviation(+/-)'] = deviation
    ada['time'] = t
    return ada

In [15]:
# RandomForestClasssifier
from sklearn.ensemble import RandomForestClassifier
def RandomForestClassifier_model(X,y):
    X_train,X_test,y_train,y_test = train_test_split_model(X,y)
    t1=time()
    rfm = RandomForestClassifier(n_estimators= 70,n_jobs=-1,random_state = 101,max_features = None, min_samples_leaf = 30)
    scores = cross_val_score(rfm,X,y, cv=20)
    scores1 = scores.mean()
    deviation = scores.std() * 2
    rfm.fit(X_train,y_train)
    models.append(rfm)
    random = calculate_recall_precision('RandomForest',y_test,rfm.predict(X_test))
    t2=time()
    t = t2-t1
    random['Accuract with cv'] = scores1
    random['Deviation(+/-)'] = deviation
    random['time'] = t    
    return random

In [16]:
# K-Nearest-NeighborClassifier
from sklearn.neighbors import KNeighborsClassifier
def knn_model(X,y):
    X_train,X_test,y_train,y_test = train_test_split_model(X,y)
    t1=time()
    knn = KNeighborsClassifier(n_neighbors=15)
    scores = cross_val_score(knn,X,y, cv=20)
    scores1 = scores.mean()
    deviation = scores.std() * 2
    knn.fit(X_train,y_train)
    models.append(knn)
    nearest = calculate_recall_precision('K-NearestNeighbours',y_test,knn.predict(X_test))
    t2=time()
    t = t2-t1
    nearest['Accuract with cv'] = scores1
    nearest['Deviation(+/-)'] = deviation
    nearest['time'] = t 
    return nearest

In [17]:
# NeuralNetwork
from sklearn.neural_network import MLPClassifier
def NeuralNetworkClassifier_model(X,y):
    X_train,X_test,y_train,y_test = train_test_split_model(X,y)
    t1=time()
    mlp = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500)
    scores = cross_val_score(mlp,X,y, cv=20)
    scores1 = scores.mean()
    deviation = scores.std() * 2
    mlp.fit(X_train,y_train)
    models.append(mlp)
    neural = calculate_recall_precision('NeuralNetwork',y_test,mlp.predict(X_test))
    t2=time()
    t = t2-t1
    neural['Accuract with cv'] = scores1
    neural['Deviation(+/-)'] = deviation
    neural['time'] = t 
    return neural

In [18]:
def all_classification_model(X,y):
    # added
    #accuracy_score(y_test,y_predict)
    names = ["LogisticRegression","NaiveBayers","SupportVectorMachine","DecisionTree","ADABoostClassifier","RandomForest","knn","NeuralNetwork"]
    logistic = logistic_regression_model(X,y)
    print('logistic done')
    naive = naive_bayers_model(X,y)
    print('naive bayers done')
    svm = support_vector_machine_model(X,y)
    print('svm done')
    decision = DecisionTreeClassifier_model(X,y)
    print('decision tree done')
    ada = AdaBoostClassifier_model(X,y)
    print('ada done')
    randomforest = RandomForestClassifier_model(X,y)
    print('random done')
    knn = knn_model(X,y)
    print('knn done')
    nn = NeuralNetworkClassifier_model(X,y)
    
    all_cl = pd.concat([logistic,naive,svm,decision,ada,randomforest,knn,nn])
    print('nn done')
    return all_cl

Read Data from csv file

In [23]:
file = "OnlineNewsPopularity.csv"
df = to_read_csv(file)
#df.set_index('url', inplace=True)
df = df.drop(['url'],axis =1)

In [24]:
df_copy = df

In [25]:
# Min Max Scaling
df_minmax = minmaxscaling(df_copy.drop([' shares'],axis=1))
# Standardization
df_std = data_standardization(df_minmax)

In [41]:
df_std.shape

(39644, 60)

In [27]:
df_std[' shares'] = df_copy[' shares']

In [28]:
df_std

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,1.757880,0.757447,-0.695210,0.032772,0.000675,0.038658,-0.607463,-0.335566,-0.426526,-0.304268,...,0.063865,-0.228941,-0.708369,-0.268895,-0.969886,0.671245,-0.975432,-1.810719,0.138920,593
1,1.757880,-0.661657,-0.618794,0.016056,0.000675,0.031479,-0.695709,-0.594963,-0.426526,-0.304268,...,-0.870968,-0.228941,1.102174,1.367424,0.078642,-0.870807,-0.269076,0.837749,-0.689658,711
2,1.757880,-0.661657,-0.712192,0.007645,0.000675,-0.007752,-0.695709,-0.594963,-0.426526,-0.304268,...,0.063865,0.981798,-1.621797,-0.957871,-0.270867,-0.870807,-0.269076,0.837749,-0.689658,1500
3,1.757880,-0.661657,-0.032933,-0.012619,0.000675,-0.007211,-0.166229,-0.854360,-0.426526,-0.304268,...,0.573773,0.174639,-0.862584,-0.268895,-0.620377,-0.870807,-0.269076,0.837749,-0.689658,1200
4,1.757880,1.230482,1.115439,-0.037655,0.000675,-0.045420,0.716237,4.074185,1.860061,-0.304268,...,-0.870968,0.981798,0.307944,0.075594,0.602906,0.531059,0.244637,-1.569949,-0.087056,505
5,1.757880,-0.188622,-0.374685,0.003316,0.000675,0.002764,-0.783956,-0.335566,-0.546872,-0.304268,...,0.573773,-0.632520,0.505184,0.420082,0.078642,1.111832,0.538188,-1.054014,0.257288,855
6,1.757880,-1.134691,0.877699,-0.036940,0.000675,-0.042680,0.892731,4.333583,1.860061,-0.304268,...,0.063865,0.981798,0.274380,0.075594,0.602906,-0.870807,-0.269076,0.837749,-0.689658,556
7,1.757880,0.757447,0.939257,-0.032563,0.000675,-0.035858,0.804484,4.333583,1.860061,-0.304268,...,0.063865,0.981798,0.131114,0.075594,0.602906,2.213297,1.614540,0.837749,1.519883,891
8,1.757880,0.284413,-0.954178,0.034621,0.000675,0.045197,-0.783956,-0.854360,-0.546872,-0.304268,...,4.270611,0.174639,1.053240,1.367424,-0.183490,-0.485294,-0.269076,0.175632,-0.689658,3600
9,1.757880,-0.188622,-0.669738,0.025037,0.000675,0.033058,-0.607463,-0.594963,-0.426526,-0.060829,...,0.063865,-1.036100,0.167775,0.075594,0.078642,-0.870807,-0.269076,0.837749,-0.689658,710


In [29]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(df_std)
labels = kmeans.labels_

In [30]:
df_std['clusters'] = labels

In [31]:
df_std.groupby(['clusters']).mean()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
clusters,,,,,,,,,,,,,,,,,,,,,
0,0.050223,-0.009419,-0.040313,-0.006771,-0.009308,-0.011210,0.206687,-0.027450,0.248970,0.151720,...,0.063361,0.054657,-0.124137,-0.069198,-0.035754,0.168787,0.077448,-0.003879,0.170501,16411.176363
1,0.099605,0.235478,-0.024881,-0.007413,-0.019100,-0.014761,0.068081,-0.317677,0.129559,0.115454,...,-0.071671,-0.180233,-0.349726,-0.136742,-0.553086,0.310305,0.091212,0.179091,0.449167,187668.965517
2,0.660575,0.126734,0.590783,-0.013493,0.000675,-0.017897,0.686822,0.312927,0.355728,0.385476,...,-0.520405,0.309165,-0.582519,-0.223441,-0.445622,-0.544038,-0.278046,0.339585,-0.258272,651516.666667
3,-0.147082,0.219740,-0.087360,-0.000388,-0.007539,-0.004693,0.383244,-0.057930,0.268288,0.157886,...,0.121846,0.074532,-0.316316,-0.235496,-0.146392,0.200831,0.167739,-0.013156,0.265737,63294.132812
4,-0.002121,-0.001164,0.002893,0.000408,0.000613,0.000705,-0.014942,0.002208,-0.016628,-0.010150,...,-0.004420,-0.003633,0.009836,0.005829,0.003612,-0.011450,-0.005730,0.000124,-0.012153,1970.031100


In [32]:
df_std.groupby(['clusters']).min()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
clusters,,,,,,,,,,,,,,,,,,,,,
0,-1.613414,-3.026830,-1.160078,-0.155714,-0.190487,-0.211094,-0.960449,-0.854360,-0.546872,-0.304268,...,-1.338384,-3.053998,-5.797463,-1.646847,-9.358111,-0.870807,-4.036308,-1.810719,-0.689658,9200
1,-1.478002,-1.607726,-1.160078,-0.155714,-0.190487,-0.211094,-0.960449,-0.854360,-0.546872,-0.304268,...,-1.338384,-3.053998,-5.014526,-1.646847,-8.309583,-0.870807,-2.906139,-1.569949,-0.689658,128500
2,-0.376028,-1.134691,-0.875639,-0.088196,0.000675,-0.097429,-0.872203,-0.594963,-0.546872,-0.060829,...,-0.870968,-1.036100,-3.448651,-1.646847,-3.066942,-0.870807,-1.399246,-1.621543,-0.689658,441000
3,-1.608745,-2.553795,-1.160078,-0.155714,-0.190487,-0.211094,-0.960449,-0.854360,-0.546872,-0.304268,...,-1.338384,-3.053998,-5.014526,-1.646847,-8.309583,-0.870807,-4.036308,-1.810719,-0.689658,39900
4,-1.618083,-3.972899,-1.160078,-0.155714,-0.190487,-0.211094,-0.960449,-0.854360,-0.546872,-0.304268,...,-1.338384,-3.053998,-5.797463,-1.646847,-9.358111,-0.870807,-4.036308,-1.810719,-0.689658,1


In [33]:
df_std.groupby(['clusters']).max()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
clusters,,,,,,,,,,,,,,,,,,,,,
0,1.757880,3.122620,16.827550,0.128324,0.000675,0.095206,12.453042,13.671880,12.811607,21.848691,...,12.684102,0.981798,2.031912,1.798034,1.127170,2.213297,3.498156,0.837749,3.729424,39800
1,1.515072,2.649586,6.937963,0.063100,0.000675,0.073327,4.246103,1.220817,1.980408,3.590758,...,2.167238,0.981798,2.031912,1.798034,1.127170,2.213297,3.498156,0.837749,3.729424,310800
2,1.505734,1.230482,4.339797,0.040100,0.000675,0.049828,1.510457,1.999008,1.619368,2.373562,...,0.063865,0.981798,0.906439,1.152119,0.602906,0.781392,0.807276,0.837749,0.636066,843300
3,1.715856,4.068690,7.519579,0.087747,0.000675,0.084266,12.100056,7.965143,6.072194,6.268588,...,7.075108,0.981798,2.031912,1.798034,1.127170,2.213297,3.498156,0.837749,3.729424,122800
4,1.757880,5.960828,15.320444,198.954458,199.000326,198.883779,25.866534,29.235709,14.857501,17.953666,...,12.684102,0.981798,2.031912,1.798034,1.127170,2.213297,3.498156,0.837749,3.729424,9100


Cluster details
0 - > (9200,39800)
1 - > (128500,310800)
2 - > (441000,843300)
3 - > (39900,122800)
4 - > (1,9100)

4 - > (1,9100)           very low
0 - > (9200,39800)       low
3 - > (39900,122800)     medium
1 - > (128500,310800)    high
2 - > (441000,843300)    very high

In [34]:
df_std = df_std.drop(' shares', axis=1)

In [35]:
df_std.shape

(39644, 60)

In [36]:
X = df_std.iloc[:,0:59]
y = df_std.iloc[:,-1:]

In [37]:
y.shape

(39644, 1)

In [38]:
X.shape

(39644, 59)

In [39]:
 y = y.values

## Feature Selection Using Boruta

In [102]:
X_boruta = X.values
y_boruta = y.values

In [105]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

In [111]:
rfc = RandomForestClassifier(n_estimators=200, n_jobs=4, class_weight='balanced', max_depth=6)
boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2)
boruta_selector.fit(X_boruta, y_boruta)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	59
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	59
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	59
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	59
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	59
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	59
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	59
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	49
Tentative: 	3
Rejected: 	7


/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	9 / 100
Confirmed: 	49
Tentative: 	3
Rejected: 	7


/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	10 / 100
Confirmed: 	49
Tentative: 	3
Rejected: 	7


...

Iteration: 	11 / 100
Confirmed: 	49
Tentative: 	3
Rejected: 	7


/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	12 / 100
Confirmed: 	50
Tentative: 	2
Rejected: 	7
Iteration: 	13 / 100
Confirmed: 	50
Tentative: 	0
Rejected: 	9


BorutaPy finished running.

Iteration: 	14 / 100
Confirmed: 	50
Tentative: 	0
Rejected: 	9


/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


BorutaPy(alpha=0.05,
     estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=6, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=169, n_jobs=4, oob_score=False,
            random_state=<mtrand.RandomState object at 0x1082aa1b0>,
            verbose=0, warm_start=False),
     max_iter=100, n_estimators='auto', perc=100,
     random_state=<mtrand.RandomState object at 0x1082aa1b0>,
     two_step=True, verbose=2)

In [112]:
print (boruta_selector.n_features_)

50


In [113]:
print (boruta_selector.ranking_)

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  5  6  7  8 10  4  8  3  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  2  1]


In [ ]:
X.column

In [114]:
len(X.columns)

59

In [115]:
print (boruta_selector.support_)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True False False False False False False
 False False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True]


In [119]:
feature_df = pd.DataFrame(X.columns.tolist(),columns=['features'])

In [120]:
feature_df['rank']=boruta_selector.ranking_

In [ ]:
 min_positive_polarity, max_positive_polarity, avg_negative_polarity, min_negative_polarity, max_negative_polarity, title_subjectivity, title_sentiment_polarity, abs_title_subjectivity, abs_title_sentiment_polarity

In [122]:
print ('\n Top %d features:' % boruta_selector.n_features_)
print (feature_df.head(boruta_selector.n_features_))


 Top 50 features:
                          features  rank
0                        timedelta     1
1                   n_tokens_title     1
2                 n_tokens_content     1
3                  n_unique_tokens     1
4                 n_non_stop_words     1
5         n_non_stop_unique_tokens     1
6                        num_hrefs     1
7                   num_self_hrefs     1
8                         num_imgs     1
9                       num_videos     1
10            average_token_length     1
11                    num_keywords     1
12       data_channel_is_lifestyle     1
13   data_channel_is_entertainment     1
14             data_channel_is_bus     1
15          data_channel_is_socmed     1
16            data_channel_is_tech     1
17           data_channel_is_world     1
18                      kw_min_min     1
19                      kw_max_min     1
20                      kw_avg_min     1
21                      kw_min_max     1
22                      kw_max_max    

In [139]:
X1 = X.drop([' min_positive_polarity', ' max_positive_polarity', ' avg_negative_polarity', ' min_negative_polarity', ' max_negative_polarity',' title_subjectivity', ' title_sentiment_polarity', ' abs_title_subjectivity', ' abs_title_sentiment_polarity'],axis=1) 

In [140]:
X1.shape

(39644, 50)

In [182]:
y

url
http://mashable.com/2013/01/07/amazon-instant-video-browser/                             1
http://mashable.com/2013/01/07/ap-samsung-sponsored-tweets/                              3
http://mashable.com/2013/01/07/apple-40-billion-app-downloads/                           3
http://mashable.com/2013/01/07/astronaut-notre-dame-bcs/                                 1
http://mashable.com/2013/01/07/att-u-verse-apps/                                         4
http://mashable.com/2013/01/07/beewi-smart-toys/                                         4
http://mashable.com/2013/01/07/bodymedia-armbandgets-update/                             4
http://mashable.com/2013/01/07/canon-poweshot-n/                                         4
http://mashable.com/2013/01/07/car-of-the-future-infographic/                            4
http://mashable.com/2013/01/07/chuck-hagel-website/                                      0
http://mashable.com/2013/01/07/cosmic-events-doomsday/                                

In [146]:
y = y.values

In [147]:
b = all_classification_model(X,y)

/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was e

TruePositive_test  3899
TrueNegative_test  7872
FlasePositive_test  61
FalseNegative_test  62
logistic done


/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was e

TruePositive_test  3022
TrueNegative_test  7700
FlasePositive_test  111
FalseNegative_test  1061
naive bayers done


/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was e

...

TruePositive_test  3779
TrueNegative_test  7717
FlasePositive_test  138
FalseNegative_test  260
random done


/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_val

TruePositive_test  3757
TrueNegative_test  7695
FlasePositive_test  136
FalseNegative_test  306
knn done


/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.p

TruePositive_test  3903
TrueNegative_test  7927
FlasePositive_test  23
FalseNegative_test  41
nn done


In [148]:
b

,Accuracy,F1_score,Model,Precision,Recall,Specificity,Accuract with cv,Deviation(+/-),time
0,0.989659,0.984472,LogisticRegression,0.984596,0.984347,0.992311,0.980329,0.038236,59.915945
0,0.901463,0.837583,NaiveBayers,0.964571,0.740142,0.985789,0.869646,0.267706,0.594845
0,0.991592,0.987348,SupportVectorMachine,0.987848,0.986849,0.993955,0.986054,0.025660,70.854863
0,0.962334,0.944209,DecisionTree,0.948936,0.939529,0.974042,0.913538,0.293186,11.475771
0,0.943081,0.914746,AdaptiveBoosting,0.968533,0.866619,0.984681,0.750581,0.281553,120.104881
0,0.966538,0.949975,RandomForest,0.964769,0.935628,0.982432,0.934841,0.186817,208.617432
0,0.962838,0.944444,K-NearestNeighbours,0.965066,0.924686,0.982633,0.957023,0.048928,311.620027
0,0.994619,0.991868,NeuralNetwork,0.994142,0.989604,0.997107,0.987947,0.037783,48.023995


In [149]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE

In [150]:
svm = LinearSVC()
# create the RFE model for the svm classifier 
# and select attributes
rfe = RFE(svm, 3)

In [151]:
rfe = rfe.fit(X,y)

/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [152]:
print(rfe.support_)

[False False False  True  True  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False]


In [153]:
print(rfe.ranking_)

[26 36 46  1  1  1 35 31 19 20 13 21 27 11  9 14  3  2 45 52 48 43 18 28
 33 32 10 40 39 25 54 47 53 55 57 51 56 42  4 17  8 16  5 12 38 24 22  6
  7 23 34 41 15 29 37 44 50 49 30]


In [ ]:
' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',' data_channel_is_tech',
       ' data_channel_is_world',
    ' LDA_00',' LDA_04'

In [ ]:
' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',' data_channel_is_tech',
       ' data_channel_is_world',
    
    
    ' rate_positive_words',
       ' rate_negative_words',' LDA_00'
    ' LDA_02'
    ' LDA_04',

In [154]:
X.columns

Index([' timedelta', ' n_tokens_title', ' n_tokens_content',
       ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',
       ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos',
       ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle',
       ' data_channel_is_entertainment', ' data_channel_is_bus',
       ' data_channel_is_socmed', ' data_channel_is_tech',
       ' data_channel_is_world', ' kw_min_min', ' kw_max_min', ' kw_avg_min',
       ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg',
       ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares',
       ' self_reference_max_shares', ' self_reference_avg_sharess',
       ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday',
       ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday',
       ' weekday_is_sunday', ' is_weekend', ' LDA_00', ' LDA_01', ' LDA_02',
       ' LDA_03', ' LDA_04', ' global_subjectivity',
       ' global_sentiment_p

In [158]:
X2 = X[[' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',' data_channel_is_entertainment', ' data_channel_is_bus',
       ' data_channel_is_socmed', ' data_channel_is_tech',
       ' data_channel_is_world',' self_reference_min_shares',' avg_negative_polarity',' rate_positive_words',
       ' rate_negative_words',' LDA_03', ' LDA_04', ' global_subjectivity',' LDA_00', ' LDA_01', ' LDA_02']]

In [160]:
b = all_classification_model(X2,y)

/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was e

TruePositive_test  3778
TrueNegative_test  7731
FlasePositive_test  153
FalseNegative_test  232
logistic done


/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was e

TruePositive_test  2908
TrueNegative_test  7711
FlasePositive_test  83
FalseNegative_test  1192
naive bayers done


/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was e

...

TruePositive_test  3776
TrueNegative_test  7661
FlasePositive_test  196
FalseNegative_test  261
random done


/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_val

TruePositive_test  3786
TrueNegative_test  7714
FlasePositive_test  171
FalseNegative_test  223
knn done


/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.p

TruePositive_test  3813
TrueNegative_test  7723
FlasePositive_test  164
FalseNegative_test  194
nn done


In [161]:
b

,Accuracy,F1_score,Model,Precision,Recall,Specificity,Accuract with cv,Deviation(+/-),time
0,0.967631,0.951517,LogisticRegression,0.961079,0.942145,0.980594,0.965496,0.027315,26.088325
0,0.892803,0.820195,NaiveBayers,0.972250,0.709268,0.989351,0.898476,0.047940,0.261091
0,0.968051,0.952213,SupportVectorMachine,0.959939,0.944611,0.979964,0.965572,0.028507,26.565346
0,0.960148,0.940898,DecisionTree,0.951097,0.930915,0.975258,0.955507,0.028288,4.076963
0,0.912141,0.872825,AdaptiveBoosting,0.893151,0.853403,0.944228,0.792513,0.195135,54.658319
0,0.961577,0.942939,RandomForest,0.950655,0.935348,0.975054,0.959998,0.031809,54.048197
0,0.966874,0.950540,K-NearestNeighbours,0.956785,0.944375,0.978313,0.964361,0.029316,28.367278
0,0.969901,0.955160,NeuralNetwork,0.958763,0.951585,0.979206,0.966883,0.026363,45.740651


In [ ]:
# Top 8 Features

In [37]:
X5 = X[[' n_unique_tokens',' n_non_stop_words',' n_non_stop_unique_tokens',' data_channel_is_tech',' data_channel_is_world',
    ' rate_positive_words',' rate_negative_words',' LDA_00',' LDA_02',' LDA_04']]

In [38]:
b1 = all_classification_model(X5,y)

/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be less than n_splits=20.
  % (min_groups, self.n_splits)), Warning)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 

TruePositive_test  0
TrueNegative_test  11147
FlasePositive_test  0
FalseNegative_test  747


ZeroDivisionError: division by zero

In [164]:
b1

,Accuracy,F1_score,Model,Precision,Recall,Specificity,Accuract with cv,Deviation(+/-),time
0,0.943921,0.918210,LogisticRegression,0.932503,0.904348,0.965050,0.940499,0.028541,10.044387
0,0.641248,0.007905,NaiveBayers,1.000000,0.003968,1.000000,0.671238,0.180967,0.197804
0,0.944930,0.919424,SupportVectorMachine,0.932618,0.906599,0.965260,0.940272,0.029099,38.980184
0,0.943165,0.917137,DecisionTree,0.928749,0.905811,0.963035,0.939389,0.030148,2.672789
0,0.554986,0.037811,AdaptiveBoosting,0.717241,0.019417,0.993729,0.564101,0.279188,35.136884
0,0.946191,0.920910,RandomForest,0.939960,0.902616,0.969354,0.942442,0.030374,27.902768
0,0.947116,0.921931,K-NearestNeighbours,0.945038,0.899927,0.972190,0.941432,0.028525,3.574392
0,0.948041,0.923572,NeuralNetwork,0.942453,0.905432,0.970656,0.942542,0.027055,37.118976


In [180]:
y

array([[1],
       [3],
       [3],
       ...,
       [1],
       [0],
       [1]], dtype=int32)

In [179]:
X7 = pd.concat([X6,y],axis=1)

TypeError: cannot concatenate object of type "<class 'numpy.ndarray'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [ ]:
corr = X6.corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)
cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [40]:
# top 5 features
X6 = X[[' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',' data_channel_is_tech',' data_channel_is_world',' LDA_00',' LDA_04']]
b3 = all_classification_model(X6,y)

/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be less than n_splits=20.
  % (min_groups, self.n_splits)), Warning)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 

TruePositive_test  0
TrueNegative_test  11147
FlasePositive_test  0
FalseNegative_test  747


ZeroDivisionError: division by zero

In [49]:
y

array([[4],
       [4],
       [4],
       ...,
       [4],
       [4],
       [4]], dtype=int32)

In [70]:
# Considering all features
a

,Accuracy,F1_score,Model,Precision,Recall,Specificity,Accuract with cv,Deviation(+/-),time
0,0.989659,0.984472,LogisticRegression,0.984596,0.984347,0.992311,0.980329,0.038236,64.771102
0,0.901463,0.837583,NaiveBayers,0.964571,0.740142,0.985789,0.869646,0.267706,0.719466
0,0.991592,0.987348,SupportVectorMachine,0.987848,0.986849,0.993955,0.986054,0.025660,64.853213
0,0.962334,0.944209,DecisionTree,0.948936,0.939529,0.974042,0.913538,0.293186,10.396585
0,0.943081,0.914746,AdaptiveBoosting,0.968533,0.866619,0.984681,0.750581,0.281553,117.653989
0,0.966538,0.949975,RandomForest,0.964769,0.935628,0.982432,0.934841,0.186817,142.853742
0,0.962838,0.944444,K-NearestNeighbours,0.965066,0.924686,0.982633,0.957023,0.048928,317.645215
0,0.993526,0.990205,NeuralNetwork,0.993871,0.986565,0.996981,0.990115,0.024506,52.727281


In [185]:
# save the model to disk
filename = 'finalized_model.sav'

In [197]:
models

[]

In [50]:
b7 = all_classification_model(X6,y)

/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 6 members, which is too few. The minimum number of members in any class cannot be less than n_splits=20.
  % (min_groups, self.n_splits)), Warning)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rishabhjain/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 

TruePositive_test  0
TrueNegative_test  11147
FlasePositive_test  0
FalseNegative_test  747


ZeroDivisionError: division by zero

In [225]:
models

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=0, warm_start=False),
 GaussianNB(priors=None),
 SVC(C=0.025, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
   max_iter=-1, probability=False, random_state=101, shrinking=True,
   tol=0.001, verbose=False),
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=101,
             splitter='best'),
 AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
           n_estimators=100, random_

# Pickling

In [226]:
pkl_filename = "all_models.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(models, file)

In [227]:
# Load from file
with open(pkl_filename, 'rb') as file:  
    pickle_model = pickle.load(file)

In [257]:
test_df = to_read_csv('test.csv')

In [258]:
test_X = test_df

In [259]:
test_X1 = data_standardization(test_X)

In [260]:
test_X2 = minmaxscaling(test_X1)

In [261]:
test_X2.shape


(32, 9)

In [264]:
test_X2=test_X2.drop([' data_channel_is_entertainment'],axis=1)

In [265]:
test_X2=test_X2.drop([' data_channel_is_lifestyle'],axis=1)

In [268]:
test_X2.shape

(32, 7)

In [237]:
y = test_df

In [269]:
a = []
#for i in model:
a = pickle_model[0].predict(test_X2) 

In [270]:
a

array([1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)